In [1]:
from image import Image
from text_recognition import text_localization, text_reco
import matplotlib.pyplot as plt
import numpy as np


In [2]:

def find_team(data, h, prox_ratio=0.005):
    if data.shape[0] == 1:
        return data
    else:
        indices=[]
        for i, val1 in enumerate(data[:, 1]):
            for j, val2 in enumerate(data[i+1:, 1]):
                distance = abs(val1-val2)
                if distance < h*prox_ratio:
                    if i not in indices:
                        indices.append(i)
                    if j+i+1 not in indices:
                        indices.append(j+i+1)
        return data[indices]

def find_players(data):
    data = data[np.argsort(data[:, 1])]
    distances=data[1:, 1] - data[:-1, 1]
    indices = np.argsort(distances)[::-1]
    distances = distances[indices]
    players=None
    team_misc=0
    i=0
    while players is None and i<indices.shape[0]:
        sep_line = indices[i]+1
        grps = [data[:sep_line], data[sep_line:]]
        for i in [0, 1]:
            if grps[i].shape[0] == 11:
                return grps[i], grps[1-i]
        i += 1

    raise ValueError("The 11 players could not be distinguished from the rest")

def find_coach_and_substitutes(data):
    data = data[np.argsort(data[:, 0], axis=0)]
    sep_line = np.argmax(data[1:, 0] - data[:-1, 0])+1
    grps = [data[:sep_line], data[sep_line:]]
    vars = [grp[:, 0].var() for grp in grps]
    index = np.argmin(np.asarray(vars))
    
    return grps[index], grps[1-index]

def separate_coach_and_substitutes(data):
    data = data[np.argsort(data[:, 1])]
    indices=[2, -2]
    dists = [data[ind, 1]-data[ind-1, 1] for ind in indices]
    sep_line = indices[np.argmax(dists)]
    grps = [data[:sep_line], data[sep_line:]]
    index = np.argmax([grp.shape[0] for grp in grps])
    return grps[index][:-1], grps[1-index][:-1]

def find_text_categories_loc(boxes, h):
    centerX = np.asarray([(endX+startX)/2 for (startX, startY, endX, endY) in boxes])
    centerY = np.asarray([h-(endY+startY)/2 for (startX, startY, endX, endY) in boxes])
    centers = np.concatenate([centerX.reshape(-1, 1), centerY.reshape(-1, 1)], axis=1)
    
    substitutes_coach, others = find_coach_and_substitutes(centers)
    subst_cent, coach_cent = separate_coach_and_substitutes(substitutes_coach)
    players_cent, others = find_players(others)
    team_cent = find_team(others, h, prox_ratio=0.005)
    
    substitutes_ind = [int(el/2) for el in np.in1d(centers, subst_cent).nonzero()[0][::2]]
    coach_ind = [int(el/2) for el in np.in1d(centers, coach_cent).nonzero()[0][::2]]
    players_ind = [int(el/2) for el in np.in1d(centers, players_cent).nonzero()[0][::2]]
    team_ind = [int(el/2) for el in np.in1d(centers, team_cent).nonzero()[0][::2]]
    #plt.figure(figsize=(10, 5))
    #plt.scatter(pts[:, 0], pts[:, 1])
    #plt.scatter(coach_cent[:, 0], coach_cent[:, 1])
    #plt.show
    return boxes[substitutes_ind], boxes[coach_ind], boxes[players_ind], boxes[team_ind]


In [3]:
path = '../Images/psg_info.png'


img = Image(path=path, name='psg_info_out')
(h, w) = img.pix_vals.shape[:2]
    #img.resize_no_distortion()
    #img.convert_to_gray()
    #img.thresholding()
    #img.export('../Images')
boxes = text_localization(img.pix_vals)

[INFO] loading EAST text detector...


In [4]:
substitutes, coach, players, team = find_text_categories_loc(boxes, h)
print(substitutes)

[[ 320  630  528  690]
 [ 320  710  504  765]
 [ 280  875  608  930]
 [ 296  790  528  845]
 [ 296  950  560 1010]
 [ 304  550  496  610]
 [ 264 1030  472 1090]]


In [5]:
#text_reco(img.pix_vals, coach, 0.5, 0.25, 'Coach')
text_reco(img.pix_vals, substitutes, 0.5, 0.25, 'Substitutes')

5 ARI

8 PAR

e AV A

eta

asa

7

ac



In [6]:


#img.display()